# Points de recharge pour véhicule électrique

Fichier consolidé des Bornes de Recharge pour Véhicules Électriques

* https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/

In [1]:
import pandas as pd 
import re

In [2]:
usecols = ['n_station', 'ad_station', 'code_insee', 'Xlongitude', 'Ylatitude', 'nbre_pdc',
       'puiss_max', 'type_prise', 'accessibilité' ] 

In [3]:
df = pd.read_csv('data/irve/bornes-irve-20191120.csv', 
                 sep=';', usecols=usecols, dtype=str )
df.head(3)

,n_station,ad_station,code_insee,Xlongitude,Ylatitude,nbre_pdc,puiss_max,type_prise,accessibilité
0,CASSIS - Parking Savon,Parking Savon 13260 CASSIS,13022,5.53811,43.213528,6,4,EF,24h/24 7j/7
1,MARSEILLE - Parking Technoptic,Parking Technoptic 13000 MARSEILLE,13055,5.431385,43.344402,4,4,EF,24h/24 7j/7
2,SALON-DE-PROVENCE - Rue André Marie Ampère,Rue André Marie Ampère 0 SALON-DE-PROVENCE,13103,5.107927,43.639222,2,22,EF - T2,24h/24 7j/7


In [4]:
df.shape

(10078, 9)

In [5]:
df = df.drop_duplicates()
df.shape

(6404, 9)

In [6]:
df.tail(3)

,n_station,ad_station,code_insee,Xlongitude,Ylatitude,nbre_pdc,puiss_max,type_prise,accessibilité
10075,VIC-SUR-CERE - Rue Du 19 Mars 1962,Rue Du 19 Mars 1962 15800 VIC-SUR-CERE,15258,2.626218,44.978116,4,4,EF,24h/24 7j/7
10076,VIC-SUR-CERE - Rue Du 19 Mars 1962,Rue Du 19 Mars 1962 15800 VIC-SUR-CERE,15258,2.626218,44.978116,4,50,CHADEMO,24h/24 7j/7
10077,VIC-SUR-CERE - Rue Du 19 Mars 1962,Rue Du 19 Mars 1962 15800 VIC-SUR-CERE,15258,2.626218,44.978116,4,50,COMBO,24h/24 7j/7


## Limpar ficheiro de multiples fontes (e encodaxes)

Limitalos caractères aceptados : 
* Os primeiros ASCII, do 0 au 7f 
* O caracter c4 (Ä) : senon erro mysql (???????)
* Os acentuados 

```
df2 = df.replace({r'[^\x00-\x7F\xC4àäâéèêëïîöôüû]+': '_'}, regex=True)
```

Enchelos NaN pra evitalo erro mysql 'na column not found'
```
df2=df2.fillna('') ```


In [7]:
#https://www.codetable.net/asciikeycodesû
df2 = df.replace({r'[^\x00-\x7F\xC4àäâéèêëïîöôüû]+': '_'}, regex=True) 
df2=df2.fillna('')  
df2.shape

(6404, 9)

In [8]:
df2.head()

,n_station,ad_station,code_insee,Xlongitude,Ylatitude,nbre_pdc,puiss_max,type_prise,accessibilité
0,CASSIS - Parking Savon,Parking Savon 13260 CASSIS,13022,5.53811,43.213528,6,4,EF,24h/24 7j/7
1,MARSEILLE - Parking Technoptic,Parking Technoptic 13000 MARSEILLE,13055,5.431385,43.344402,4,4,EF,24h/24 7j/7
2,SALON-DE-PROVENCE - Rue André Marie Ampère,Rue André Marie Ampère 0 SALON-DE-PROVENCE,13103,5.107927,43.639222,2,22,EF - T2,24h/24 7j/7
3,CORNILLON-CONFOUX - Rue Des Ferrages,Rue Des Ferrages 13250 CORNILLON-CONFOUX,13029,5.071913,43.565413,2,22,EF - T2,24h/24 7j/7
4,MARSEILLE - Parking Relais Einstein,Parking Relais Einstein 13000 MARSEILLE,13055,5.448569,43.345406,2,4,EF,24h/24 7j/7


### EPCI

In [9]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='123',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

In [10]:
epci_data = pd.read_csv('data/insee/communes_cp_insee_epci_geo.csv', dtype=str)
epci_data = epci_data.drop_duplicates()

In [11]:
#epci 

epci = pd.merge(df2, epci_data, how='left', left_on="code_insee", right_on='codgeo')

In [12]:
epci =epci.fillna('')

In [13]:
epci.shape

(6900, 17)

In [14]:
epci.head(3)

,n_station,ad_station,code_insee,Xlongitude,Ylatitude,nbre_pdc,puiss_max,type_prise,accessibilité,codepostal,coordonnees_gps,codgeo,libcommune,epci,libepci,dep,reg
0,CASSIS - Parking Savon,Parking Savon 13260 CASSIS,13022,5.53811,43.213528,6,4,EF,24h/24 7j/7,13260,"43.2230470065,5.55117810991",13022,Cassis,200054807,Métropole d'Aix-Marseille-Provence,13,93
1,MARSEILLE - Parking Technoptic,Parking Technoptic 13000 MARSEILLE,13055,5.431385,43.344402,4,4,EF,24h/24 7j/7,,,,,,,,
2,SALON-DE-PROVENCE - Rue André Marie Ampère,Rue André Marie Ampère 0 SALON-DE-PROVENCE,13103,5.107927,43.639222,2,22,EF - T2,24h/24 7j/7,13300,"43.6462885238,5.06814649576",13103,Salon-de-Provence,200054807,Métropole d'Aix-Marseille-Provence,13,93


In [15]:
epci2 = epci.drop(['codepostal', 'coordonnees_gps', 'codgeo', 'libcommune', 'dep', 'reg'], axis=1)

In [16]:
# Insertion BDD 

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `recharge` "
           "(n_station, ad_station, code_insee, epci, Xlongitude, Ylatitude, "
           " nbre_pdc, puiss_max, type_prise, accessibilite)"
           " VALUES (%s, %s, %s, %s, %s, %s, "
           " %s, %s, %s, %s)"
          )
    print(sql)
    
    for i, r in epci2.iterrows():
        cursor.execute(sql, 
            (
            r['n_station'],
            r['ad_station'],
            r['code_insee'],
            r['epci'],
            r['Xlongitude'],
            r['Ylatitude'],
            r['nbre_pdc'],
            r['puiss_max'],
            r['type_prise'],
            r['accessibilité'] 
            )
        )

connection.commit() 

INSERT INTO `recharge` (n_station, ad_station, code_insee, epci, Xlongitude, Ylatitude,  nbre_pdc, puiss_max, type_prise, accessibilite) VALUES (%s, %s, %s, %s, %s, %s,  %s, %s, %s, %s)
